In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import scipy as sp
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer
import operator
import heapq
import warnings
import math
warnings.filterwarnings('ignore')

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/unsuplearnpred/sample_submission.csv
/kaggle/input/unsuplearnpred/movies.csv
/kaggle/input/unsuplearnpred/imdb_data.csv
/kaggle/input/unsuplearnpred/genome_tags.csv
/kaggle/input/unsuplearnpred/genome_scores.csv
/kaggle/input/unsuplearnpred/train.csv
/kaggle/input/unsuplearnpred/test.csv
/kaggle/input/unsuplearnpred/tags.csv
/kaggle/input/unsuplearnpred/links.csv


In [2]:
train_df = pd.read_csv('../input/unsuplearnpred/train.csv')
test_df = pd.read_csv('../input/unsuplearnpred/test.csv')

In [ ]:
train_df.head()

# **EDA**

In [ ]:
# Top 10 movies
TopTen = train_df.query("rating == 5")
print(TopTen["movieId"].value_counts().head(10))

In [ ]:
count_df = train_df.copy()
count_df['Ratings_per_user'] = count_df.groupby('userId')['userId'].transform('count')
count_df.head()

# **Data Preprocessing**

In [ ]:
train_df.drop(['timestamp'], axis = 1, inplace = True)

In [ ]:
#sampling test data
sample_df = train_df.sample(n=5000, random_state = 42)

In [ ]:
pvt_table = pd.pivot_table(sample_df, values = 'rating', columns = 'movieId', index = 'userId', fill_value = 0)
pvt_table_trans = pvt_table.T

In [ ]:
pvt_table

In [ ]:
pvt_table_trans

In [ ]:
user_similarity = cosine_similarity(pvt_table)
user_similarity

In [ ]:
user_sim_df = pd.DataFrame(user_similarity,
                          index = pvt_table_trans.columns,
                          columns = pvt_table_trans.columns)
user_sim_df

In [ ]:
def movie_rating_test(reference_user, reference_movie, k):
    
    #extract similar user from user similarity matrix and sort
    users_sorted = pd.DataFrame(user_sim_df.loc[:,reference_user]).sort_values(by = reference_user, ascending = False)
    #remove reference_user
    most_similar_users = users_sorted.drop(users_sorted.iloc[0:]).rename(columns = {reference_user:'sim_to_current_user'})
    #select k amount of observation
    k_most_similar_users = most_similar_users[:k]
    #select movie rating of reference_movie by similar users
    most_similar_users_rating_of_current_movie = pd.DataFrame(pvt_table_trans.loc[reference_movie,k_most_similar_users.index]).rename(columns = {reference_movie: 'most_similar_users_rating_of_current_movie'})
    #create new dataframe
    testing_df = k_most_similar_users.merge(most_similar_users_rating_of_current_movie, on='userId', how='left')
    #add simliarity_x_rating
    testing_df['simliarity_x_rating'] = testing_df['sim_to_current_user'] * testing_df['most_similar_users_rating_of_current_movie']
    
    
    #user_rating = round(user_sim_df['similarity_x_rating'].sum() / user_sim_df['user_similarity'].sum())
    user_rating = round(testing_df['simliarity_x_rating'].sum() / testing_df['sim_to_current_user'].sum() * 2) / 2
    
    #return testing_df
    #return most_similar_users_rating_of_current_movie
    #return user_rating
    return user_rating

In [ ]:
movie_rating_test(104339,356,10)

In [ ]:
def avg_rating_user(reference_user):
    #find the mean rating value of each individual user
    mean_rating = train_df.groupby('userId').mean()
    #rating = round(mean_rating.loc[reference_user,'rating'])
    rating = round(mean_rating.loc[reference_user,'rating'] * 2) / 2
    return rating

In [ ]:
avg_rating_user(21)

In [ ]:
def avg_rating_movie(reference_movie):
    #find mean rating of movies
    movie_mean = train_df.groupby('movieId').mean()
    rating = round(movie_mean.loc[reference_movie,'rating'] * 2) / 2
    return rating

In [ ]:
avg_rating_movie(204698)

In [ ]:
def movie_rating(reference_user, reference_movie):
    #6a - Isolate the userid and the movieid (store them in variables user_id and movie_id)
    sample_user_id = pd.DataFrame(user_sim_df.index)
    sample_movie_id = pd.DataFrame(pvt_table_trans.index)
    
    if reference_user not in sample_user_id['userId'].values:
        if reference_movie in sample_movie_id['movieId'].values:
            return avg_rating_movie(reference_movie) #user_sample fasle movie_sample true - avg_rating_movie
        elif reference_movie in train_df['movieId'].values: #user_sample false movie-sample false
            return avg_rating_movie(reference_movie) #movie_train true - avg_rating_movie
        elif reference_user in train_df['userId'].values:#movie_train false
            return avg_rating_user(reference_user) #user_train true - avg_rating_user
        else:
            return round(train_df['rating'].mean() * 2) / 2  #user_train false - avg_rating_all_movies
        
    if reference_movie not in sample_movie_id['movieId'].values:
        return avg_rating_user(reference_user) #user_sample true movie-sample false - avg_rating_user
        
    else:
        # user true movie true
        k = 10

        #extract similar user from user similarity matrix and sort
        users_sorted = pd.DataFrame(user_sim_df.loc[:,reference_user]).sort_values(by = reference_user, ascending = False)
        #remove reference_user
        most_similar_users = users_sorted.drop(users_sorted.iloc[0:]).rename(columns = {reference_user:'sim_to_current_user'})
        #select k amount of observation
        k_most_similar_users = most_similar_users[:k]
        #select movie rating of reference_movie by similar users
        most_similar_users_rating_of_current_movie = pd.DataFrame(pvt_table_trans.loc[reference_movie,k_most_similar_users.index]).rename(columns = {reference_movie: 'most_similar_users_rating_of_current_movie'})
        #create new dataframe
        testing_df = k_most_similar_users.merge(most_similar_users_rating_of_current_movie, on='userId', how='left')
        #add simliarity_x_rating
        testing_df['simliarity_x_rating'] = testing_df['sim_to_current_user'] * testing_df['most_similar_users_rating_of_current_movie']
        if testing_df['sim_to_current_user'].sum() == 0:
            return avg_rating_movie(reference_movie)
        else:
            user_rating = round(testing_df['simliarity_x_rating'].sum() / testing_df['sim_to_current_user'].sum() * 2) / 2
            if user_rating == 0:
                return avg_rating_user(reference_user)
            else:
                return user_rating

In [ ]:
movie_rating(3,778) # false true

In [ ]:
movie_rating(3,207404) # false false

In [ ]:
movie_rating(21,207404) # true false

In [ ]:
movie_rating(104339,356) # true true

In [ ]:
rating_list = []

for idx in test_df.index:
    reference_user = test_df.iloc[idx][0]
    reference_movie = test_df.iloc[idx][1]
    
    rating = movie_rating(reference_user, reference_movie)
    
    rating_list.append(rating)